# 🔬 AI Clustering & Insight Dashboard

## Challenge 2 : Clustering Intelligent avec Visualisations Interactives

Ce notebook propose une analyse complète de clustering avec :
- **Algorithmes** : K-Means, DBSCAN, GMM, OPTICS, K-Medoids
- **Réduction de dimensionnalité** : PCA, t-SNE, UMAP
- **Métriques** : Silhouette, Davies-Bouldin, Calinski-Harabasz
- **Visualisations** : Plotly interactif, Radar charts, Heatmaps
- **IA** : Analyse automatique des profils de clusters

## 📦 Installation des bibliothèques

In [ ]:
# Installation des bibliothèques nécessaires
!pip install umap-learn scikit-learn-extra plotly pandas numpy matplotlib seaborn -q

print("✅ Installation terminée!")

## 📚 Importation des bibliothèques

In [ ]:
# Imports standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Plotly pour visualisations interactives
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Réduction de dimensionnalité
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

# Algorithmes de clustering
from sklearn.cluster import KMeans, DBSCAN, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn_extra.cluster import KMedoids

# Métriques d'évaluation
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import silhouette_samples

# Pour charger des fichiers depuis Google Drive
from google.colab import files

print("✅ Bibliothèques importées avec succès!")

## 📊 Chargement des données

Vous pouvez charger vos données de 3 façons :
1. **Upload direct** depuis votre ordinateur
2. **Google Drive** (monter votre drive)
3. **URL** (lien direct vers un CSV)

In [ ]:
# Option 1: Upload depuis votre ordinateur
print("📁 Sélectionnez votre fichier CSV...")
uploaded = files.upload()

# Charger le premier fichier uploadé
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print(f"\n✅ Fichier chargé: {filename}")
print(f"📏 Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
print("\n📋 Aperçu des données:")
display(df.head())

In [ ]:
# Option 2: Depuis Google Drive (décommentez si nécessaire)
# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/votre_fichier.csv')

# Option 3: Depuis une URL (décommentez et modifiez l'URL)
# url = 'https://example.com/data.csv'
# df = pd.read_csv(url)

## 🔍 Exploration des données

In [ ]:
# Informations générales
print("📊 INFORMATIONS SUR LE DATASET")
print("=" * 50)
print(f"Nombre de lignes: {df.shape[0]}")
print(f"Nombre de colonnes: {df.shape[1]}")
print(f"\nMémoire utilisée: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n📋 TYPES DE DONNÉES")
print("=" * 50)
print(df.dtypes.value_counts())

print("\n⚠️ VALEURS MANQUANTES")
print("=" * 50)
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Colonnes': missing.index,
    'Valeurs manquantes': missing.values,
    'Pourcentage': missing_pct.values
})
missing_df = missing_df[missing_df['Valeurs manquantes'] > 0].sort_values('Valeurs manquantes', ascending=False)

if len(missing_df) > 0:
    display(missing_df)
else:
    print("✅ Aucune valeur manquante!")

print("\n📈 STATISTIQUES DESCRIPTIVES")
print("=" * 50)
display(df.describe())

## 🛠️ Prétraitement des données

In [ ]:
def preprocess_data(df):
    """
    Prétraite les données:
    - Gère les valeurs manquantes
    - Encode les variables catégorielles
    - Sépare les colonnes numériques et catégorielles
    """
    df_processed = df.copy()
    
    # Identifier les colonnes numériques et catégorielles
    numeric_cols = df_processed.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = df_processed.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"📊 Colonnes numériques: {len(numeric_cols)}")
    print(f"📝 Colonnes catégorielles: {len(categorical_cols)}")
    
    # Gérer les valeurs manquantes (colonnes numériques)
    if len(numeric_cols) > 0:
        imputer_num = SimpleImputer(strategy='median')
        df_processed[numeric_cols] = imputer_num.fit_transform(df_processed[numeric_cols])
    
    # Gérer les valeurs manquantes (colonnes catégorielles)
    if len(categorical_cols) > 0:
        imputer_cat = SimpleImputer(strategy='most_frequent')
        df_processed[categorical_cols] = imputer_cat.fit_transform(df_processed[categorical_cols])
    
    # Encoder les variables catégorielles
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df_processed[col + '_encoded'] = le.fit_transform(df_processed[col].astype(str))
        label_encoders[col] = le
    
    return df_processed, numeric_cols, categorical_cols, label_encoders

# Appliquer le prétraitement
df_processed, numeric_cols, categorical_cols, label_encoders = preprocess_data(df)

print("\n✅ Prétraitement terminé!")
print(f"\n📋 Colonnes numériques: {numeric_cols[:5]}..." if len(numeric_cols) > 5 else f"\n📋 Colonnes numériques: {numeric_cols}")
if len(categorical_cols) > 0:
    print(f"📋 Colonnes catégorielles: {categorical_cols}")

## 🎯 Configuration de l'analyse

Sélectionnez les colonnes à utiliser pour le clustering

In [ ]:
# Afficher toutes les colonnes disponibles
available_cols = numeric_cols.copy()
for col in categorical_cols:
    available_cols.append(col + '_encoded')

print("📊 Colonnes disponibles pour le clustering:")
for i, col in enumerate(available_cols, 1):
    print(f"{i}. {col}")

# Sélection automatique (toutes les colonnes numériques) ou manuelle
# Option 1: Utiliser toutes les colonnes numériques
selected_cols = numeric_cols.copy()

# Option 2: Sélection manuelle (décommentez et modifiez)
# selected_cols = ['colonne1', 'colonne2', 'colonne3']

print(f"\n✅ Colonnes sélectionnées ({len(selected_cols)}): {selected_cols}")

# Préparer les données pour le clustering
X = df_processed[selected_cols].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"\n📏 Shape des données: {X_scaled.shape}")
print("✅ Données normalisées et prêtes pour le clustering!")

## 📉 Réduction de dimensionnalité

Trois méthodes disponibles: PCA, t-SNE, UMAP

In [ ]:
# ========== CONFIGURATION ==========
# Choisissez la méthode: 'PCA', 't-SNE', ou 'UMAP'
REDUCTION_METHOD = 'PCA'
N_COMPONENTS = 2  # 2 pour 2D, 3 pour 3D

# Paramètres t-SNE
PERPLEXITY = 30

# Paramètres UMAP
N_NEIGHBORS = 15
MIN_DIST = 0.1
# ===================================

print(f"🔄 Réduction de dimensionnalité avec {REDUCTION_METHOD}...")

if REDUCTION_METHOD == 'PCA':
    reducer = PCA(n_components=N_COMPONENTS, random_state=42)
    X_reduced = reducer.fit_transform(X_scaled)
    variance_explained = reducer.explained_variance_ratio_
    print(f"✅ Variance expliquée: {sum(variance_explained)*100:.2f}%")
    for i, var in enumerate(variance_explained, 1):
        print(f"   - Composante {i}: {var*100:.2f}%")

elif REDUCTION_METHOD == 't-SNE':
    reducer = TSNE(n_components=N_COMPONENTS, perplexity=PERPLEXITY, random_state=42, n_iter=1000)
    X_reduced = reducer.fit_transform(X_scaled)
    print(f"✅ t-SNE terminé (perplexity={PERPLEXITY})")

elif REDUCTION_METHOD == 'UMAP':
    reducer = umap.UMAP(n_components=N_COMPONENTS, n_neighbors=N_NEIGHBORS, min_dist=MIN_DIST, random_state=42)
    X_reduced = reducer.fit_transform(X_scaled)
    print(f"✅ UMAP terminé (n_neighbors={N_NEIGHBORS}, min_dist={MIN_DIST})")

print(f"\n📏 Shape après réduction: {X_reduced.shape}")

## 🎲 Clustering - Test de TOUS les algorithmes

Nous allons tester automatiquement tous les algorithmes disponibles avec différents paramètres

In [ ]:
# ========== CONFIGURATION POUR TOUS LES ALGORITHMES ==========
print("🔄 Test de TOUS les algorithmes de clustering...\n")
print("=" * 70)

# Définir tous les algorithmes à tester
all_algorithms = {
    'K-Means (k=3)': {
        'model': KMeans(n_clusters=3, random_state=42, n_init=10),
        'name': 'K-Means',
        'params': 'k=3'
    },
    'K-Means (k=4)': {
        'model': KMeans(n_clusters=4, random_state=42, n_init=10),
        'name': 'K-Means',
        'params': 'k=4'
    },
    'K-Means (k=5)': {
        'model': KMeans(n_clusters=5, random_state=42, n_init=10),
        'name': 'K-Means',
        'params': 'k=5'
    },
    'DBSCAN (eps=0.3)': {
        'model': DBSCAN(eps=0.3, min_samples=5),
        'name': 'DBSCAN',
        'params': 'eps=0.3, min_samples=5'
    },
    'DBSCAN (eps=0.5)': {
        'model': DBSCAN(eps=0.5, min_samples=5),
        'name': 'DBSCAN',
        'params': 'eps=0.5, min_samples=5'
    },
    'DBSCAN (eps=0.7)': {
        'model': DBSCAN(eps=0.7, min_samples=5),
        'name': 'DBSCAN',
        'params': 'eps=0.7, min_samples=5'
    },
    'GMM (k=3)': {
        'model': GaussianMixture(n_components=3, random_state=42),
        'name': 'GMM',
        'params': 'k=3'
    },
    'GMM (k=4)': {
        'model': GaussianMixture(n_components=4, random_state=42),
        'name': 'GMM',
        'params': 'k=4'
    },
    'GMM (k=5)': {
        'model': GaussianMixture(n_components=5, random_state=42),
        'name': 'GMM',
        'params': 'k=5'
    },
    'OPTICS (min_samples=5)': {
        'model': OPTICS(min_samples=5, max_eps=2.0),
        'name': 'OPTICS',
        'params': 'min_samples=5, max_eps=2.0'
    },
    'OPTICS (min_samples=10)': {
        'model': OPTICS(min_samples=10, max_eps=2.0),
        'name': 'OPTICS',
        'params': 'min_samples=10, max_eps=2.0'
    },
    'K-Medoids (k=3)': {
        'model': KMedoids(n_clusters=3, random_state=42),
        'name': 'K-Medoids',
        'params': 'k=3'
    },
    'K-Medoids (k=4)': {
        'model': KMedoids(n_clusters=4, random_state=42),
        'name': 'K-Medoids',
        'params': 'k=4'
    },
}

# Stocker tous les résultats
all_results = {}
comparison_results = []

# Tester chaque algorithme
for algo_name, algo_info in all_algorithms.items():
    try:
        print(f"\n🔄 Test de {algo_name}...")
        
        # Appliquer le clustering
        model = algo_info['model']
        test_labels = model.fit_predict(X_scaled)
        
        # Calculer les centres si disponibles
        if hasattr(model, 'cluster_centers_'):
            centers = model.cluster_centers_
        elif hasattr(model, 'means_'):
            centers = model.means_
        else:
            centers = None
        
        # Calculer les métriques
        sil, db, ch = calculate_metrics(X_scaled, test_labels)
        
        # Statistiques des clusters
        unique_labels = set(test_labels)
        n_clusters = len([l for l in unique_labels if l != -1])
        n_noise = sum(test_labels == -1)
        
        # Stocker les résultats
        all_results[algo_name] = {
            'labels': test_labels,
            'centers': centers,
            'silhouette': sil,
            'davies_bouldin': db,
            'calinski': ch,
            'n_clusters': n_clusters,
            'n_noise': n_noise
        }
        
        # Pour le tableau comparatif
        comparison_results.append({
            'Algorithme': algo_info['name'],
            'Paramètres': algo_info['params'],
            'N_Clusters': n_clusters,
            'N_Outliers': n_noise,
            'Silhouette': f"{sil:.4f}" if sil else 'N/A',
            'Davies-Bouldin': f"{db:.4f}" if db else 'N/A',
            'Calinski-Harabasz': f"{ch:.1f}" if ch else 'N/A'
        })
        
        print(f"   ✅ Terminé: {n_clusters} clusters, Silhouette={sil:.4f if sil else 'N/A'}")
        
    except Exception as e:
        print(f"   ⚠️ Erreur: {str(e)}")

print(f"\n{'='*70}")
print(f"✅ Test de {len(all_results)} algorithmes terminé!")

# Créer le DataFrame comparatif
comparison_df = pd.DataFrame(comparison_results)

print("\n📊 TABLEAU COMPARATIF DE TOUS LES ALGORITHMES")
print("=" * 90)
display(comparison_df)

# Trouver le meilleur algorithme basé sur le Silhouette Score
valid_results = [(name, info) for name, info in all_results.items() if info['silhouette'] is not None]
if valid_results:
    best_algo = max(valid_results, key=lambda x: x[1]['silhouette'])
    best_name = best_algo[0]
    best_info = best_algo[1]
    
    print(f"\n🏆 MEILLEUR ALGORITHME (Silhouette Score):")
    print(f"   • Algorithme: {best_name}")
    print(f"   • Silhouette Score: {best_info['silhouette']:.4f}")
    print(f"   • Davies-Bouldin: {best_info['davies_bouldin']:.4f}")
    print(f"   • Nombre de clusters: {best_info['n_clusters']}")
    
    # Utiliser le meilleur algorithme pour les visualisations suivantes
    labels = best_info['labels']
    centers = best_info['centers']
    silhouette = best_info['silhouette']
    davies_bouldin = best_info['davies_bouldin']
    calinski = best_info['calinski']
    CLUSTERING_ALGORITHM = best_name
    
    print(f"\n💡 Les visualisations suivantes utiliseront: {best_name}")
else:
    # Fallback sur le premier algorithme
    first_algo = list(all_results.keys())[0]
    labels = all_results[first_algo]['labels']
    centers = all_results[first_algo]['centers']
    silhouette = all_results[first_algo]['silhouette']
    davies_bouldin = all_results[first_algo]['davies_bouldin']
    calinski = all_results[first_algo]['calinski']
    CLUSTERING_ALGORITHM = first_algo
    print(f"\n⚠️ Utilisation de {first_algo} pour les visualisations")

## 📊 Visualisation comparative des algorithmes

In [ ]:
# Graphique comparatif des Silhouette Scores
fig = go.Figure()

silhouette_values = []
algo_names = []
colors = []

for result in comparison_results:
    try:
        sil_val = float(result['Silhouette'])
        silhouette_values.append(sil_val)
        algo_names.append(f"{result['Algorithme']}\n{result['Paramètres']}")
        
        # Couleur selon la qualité
        if sil_val > 0.5:
            colors.append('#2ecc71')  # Vert - Excellent
        elif sil_val > 0.3:
            colors.append('#f39c12')  # Orange - Bon
        else:
            colors.append('#e74c3c')  # Rouge - Faible
    except:
        pass

fig.add_trace(go.Bar(
    x=algo_names,
    y=silhouette_values,
    marker_color=colors,
    text=[f'{v:.3f}' for v in silhouette_values],
    textposition='outside',
))

fig.update_layout(
    title='🎯 Comparaison des Silhouette Scores - Tous les algorithmes',
    xaxis_title='Algorithme',
    yaxis_title='Silhouette Score',
    plot_bgcolor='white',
    height=600,
    xaxis={'tickangle': -45},
    showlegend=False
)

fig.add_hline(y=0.5, line_dash="dash", line_color="green", 
              annotation_text="Excellente séparation (>0.5)", 
              annotation_position="right")
fig.add_hline(y=0.3, line_dash="dash", line_color="orange", 
              annotation_text="Bonne séparation (>0.3)", 
              annotation_position="right")

fig.show()

# Graphique du nombre de clusters trouvés
fig2 = go.Figure()

cluster_counts = [result['N_Clusters'] for result in comparison_results]
algo_labels = [f"{result['Algorithme']}\n{result['Paramètres']}" for result in comparison_results]

fig2.add_trace(go.Bar(
    x=algo_labels,
    y=cluster_counts,
    marker_color='#3498db',
    text=cluster_counts,
    textposition='outside',
))

fig2.update_layout(
    title='📊 Nombre de clusters trouvés par algorithme',
    xaxis_title='Algorithme',
    yaxis_title='Nombre de clusters',
    plot_bgcolor='white',
    height=600,
    xaxis={'tickangle': -45},
    showlegend=False
)

fig2.show()

print("\n📈 Légende des couleurs (Silhouette Score):")
print("   🟢 Vert: Excellente séparation (> 0.5)")
print("   🟠 Orange: Bonne séparation (0.3 - 0.5)")
print("   🔴 Rouge: Faible séparation (< 0.3)")

## 🎨 Visualisation de tous les algorithmes côte à côte

In [ ]:
# Créer une grille de visualisations pour comparer visuellement tous les algorithmes
# Sélectionner les meilleurs représentants de chaque algorithme

best_per_algo = {}
for name, info in all_results.items():
    algo_base = name.split('(')[0].strip()
    if algo_base not in best_per_algo or (info['silhouette'] and 
        (best_per_algo[algo_base]['silhouette'] is None or 
         info['silhouette'] > best_per_algo[algo_base]['silhouette'])):
        best_per_algo[algo_base] = {**info, 'full_name': name}

# Créer les subplots
n_algos = len(best_per_algo)
n_cols = 3
n_rows = (n_algos + n_cols - 1) // n_cols

if N_COMPONENTS == 2:
    fig = make_subplots(
        rows=n_rows, cols=n_cols,
        subplot_titles=[name for name in best_per_algo.keys()],
        horizontal_spacing=0.1,
        vertical_spacing=0.15
    )
    
    for idx, (algo_name, algo_info) in enumerate(best_per_algo.items()):
        row = idx // n_cols + 1
        col = idx % n_cols + 1
        
        algo_labels = algo_info['labels']
        
        # Créer un scatter pour chaque cluster
        for cluster_id in set(algo_labels):
            mask = algo_labels == cluster_id
            cluster_name = f'Cluster {cluster_id}' if cluster_id != -1 else 'Bruit'
            
            fig.add_trace(
                go.Scatter(
                    x=X_reduced[mask, 0],
                    y=X_reduced[mask, 1],
                    mode='markers',
                    name=cluster_name,
                    marker=dict(size=5, opacity=0.6),
                    showlegend=(idx == 0)  # Légende seulement pour le premier
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        height=400 * n_rows,
        title_text=f"Comparaison visuelle de tous les algorithmes - {REDUCTION_METHOD}",
        showlegend=True
    )
    
    fig.show()
    
print(f"\n📊 {len(best_per_algo)} algorithmes visualisés")
print("\n🏆 Meilleur algorithme de chaque type:")
for algo_name, algo_info in best_per_algo.items():
    sil = algo_info['silhouette']
    print(f"   • {algo_name}: Silhouette={sil:.4f if sil else 'N/A'}, Clusters={algo_info['n_clusters']}")

## 📊 Métriques d'évaluation du meilleur algorithme

In [ ]:
# Les métriques sont déjà calculées pour le meilleur algorithme
print(f"📊 MÉTRIQUES DE QUALITÉ - {CLUSTERING_ALGORITHM}")
print("=" * 70)

if silhouette is not None:
    print(f"\n🎯 Silhouette Score: {silhouette:.4f}")
    if silhouette > 0.7:
        print("   → Excellente séparation des clusters! ⭐⭐⭐")
    elif silhouette > 0.5:
        print("   → Bonne séparation des clusters ⭐⭐")
    elif silhouette > 0.3:
        print("   → Séparation acceptable ⭐")
    else:
        print("   → Faible séparation, essayez d'autres paramètres")
    
    print(f"\n📉 Davies-Bouldin Index: {davies_bouldin:.4f}")
    if davies_bouldin < 1.0:
        print("   → Excellente qualité de clustering! ⭐⭐⭐")
    elif davies_bouldin < 1.5:
        print("   → Bonne qualité de clustering ⭐⭐")
    else:
        print("   → Qualité moyenne, essayez d'autres paramètres ⭐")
    
    print(f"\n📈 Calinski-Harabasz Score: {calinski:.2f}")
    print("   → Plus le score est élevé, meilleur est le clustering")
    
    # Graphique radar des métriques normalisées
    print("\n📊 Comparaison visuelle des 3 métriques:")
    
    # Normaliser les métriques pour le radar (0-1)
    sil_norm = (silhouette + 1) / 2  # De [-1,1] à [0,1]
    db_norm = max(0, 1 - davies_bouldin / 3)  # Inverser et normaliser
    ch_norm = min(1, calinski / 1000)  # Normaliser (supposant max ~1000)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=[sil_norm, db_norm, ch_norm, sil_norm],
        theta=['Silhouette\n(Séparation)', 'Davies-Bouldin\n(Compacité)', 
               'Calinski-Harabasz\n(Variance)', 'Silhouette\n(Séparation)'],
        fill='toself',
        name=CLUSTERING_ALGORITHM,
        line=dict(color='#1f77b4', width=2)
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 1])
        ),
        showlegend=True,
        title=f'Qualité du clustering - {CLUSTERING_ALGORITHM}',
        height=500
    )
    
    fig.show()
else:
    print("⚠️ Impossible de calculer les métriques (pas assez de clusters valides)")

## 📈 Visualisation: Scatter Plot 2D/3D

In [ ]:
# Créer un DataFrame pour Plotly
plot_df = pd.DataFrame()
plot_df['Cluster'] = [f'Cluster {l}' if l != -1 else 'Bruit' for l in labels]

if N_COMPONENTS == 2:
    plot_df['Composante 1'] = X_reduced[:, 0]
    plot_df['Composante 2'] = X_reduced[:, 1]
    
    fig = px.scatter(
        plot_df,
        x='Composante 1',
        y='Composante 2',
        color='Cluster',
        title=f'Visualisation des Clusters - {CLUSTERING_ALGORITHM} + {REDUCTION_METHOD}',
        color_discrete_sequence=px.colors.qualitative.Bold,
        width=900,
        height=600
    )
    
    fig.update_traces(marker=dict(size=8, opacity=0.7, line=dict(width=0.5, color='white')))
    fig.update_layout(
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(size=12),
        legend=dict(title='Clusters', orientation='v', yanchor='top', y=1, xanchor='left', x=1.02)
    )
    
elif N_COMPONENTS == 3:
    plot_df['Composante 1'] = X_reduced[:, 0]
    plot_df['Composante 2'] = X_reduced[:, 1]
    plot_df['Composante 3'] = X_reduced[:, 2]
    
    fig = px.scatter_3d(
        plot_df,
        x='Composante 1',
        y='Composante 2',
        z='Composante 3',
        color='Cluster',
        title=f'Visualisation 3D des Clusters - {CLUSTERING_ALGORITHM} + {REDUCTION_METHOD}',
        color_discrete_sequence=px.colors.qualitative.Bold,
        width=900,
        height=700
    )
    
    fig.update_traces(marker=dict(size=5, opacity=0.7))

fig.show()

## 📊 Distribution des clusters

In [ ]:
# Graphique en barres
unique_labels, counts = np.unique(labels, return_counts=True)
cluster_names = [f'Cluster {l}' if l != -1 else 'Bruit' for l in unique_labels]

fig = px.bar(
    x=cluster_names,
    y=counts,
    title='Distribution des points par cluster',
    labels={'x': 'Cluster', 'y': 'Nombre de points'},
    color=counts,
    color_continuous_scale='Blues',
    text=counts
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=500
)

fig.show()

# Tableau des statistiques
stats_df = pd.DataFrame({
    'Cluster': cluster_names,
    'Nombre de points': counts,
    'Pourcentage': [f"{(c/len(labels))*100:.1f}%" for c in counts]
})

print("\n📋 Tableau de distribution:")
display(stats_df)

## 🎯 Profils des clusters - Radar Charts

In [ ]:
# Créer un DataFrame avec les clusters
df_with_clusters = df_processed.copy()
df_with_clusters['Cluster'] = labels

# Sélectionner les features pour le radar chart (max 6 pour la lisibilité)
radar_features = selected_cols[:min(6, len(selected_cols))]

# Créer un radar chart pour chaque cluster (max 4 clusters pour éviter surcharge)
unique_labels_sorted = sorted([l for l in set(labels) if l != -1])[:4]

for cluster_label in unique_labels_sorted:
    cluster_data = df_with_clusters[df_with_clusters['Cluster'] == cluster_label][radar_features]
    overall_data = df_processed[radar_features]
    
    # Calculer les moyennes normalisées
    cluster_means = []
    overall_means = []
    
    for col in radar_features:
        cluster_mean = cluster_data[col].mean()
        overall_mean = overall_data[col].mean()
        overall_std = overall_data[col].std()
        
        if overall_std > 0:
            cluster_normalized = (cluster_mean - overall_mean) / overall_std
        else:
            cluster_normalized = 0
        
        cluster_means.append(cluster_normalized)
        overall_means.append(0)
    
    # Créer le radar chart
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=cluster_means + [cluster_means[0]],
        theta=radar_features + [radar_features[0]],
        fill='toself',
        name=f'Cluster {cluster_label}',
        line=dict(color='#1f77b4', width=2)
    ))
    
    fig.add_trace(go.Scatterpolar(
        r=overall_means + [overall_means[0]],
        theta=radar_features + [radar_features[0]],
        fill='toself',
        name='Moyenne générale',
        line=dict(dash='dash', color='gray')
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[-3, 3], showticklabels=True)
        ),
        showlegend=True,
        title=f'Profil du Cluster {cluster_label} (Scores Z standardisés)',
        height=500
    )
    
    fig.show()

## 🔥 Heatmap des caractéristiques par cluster

In [ ]:
# Calculer les moyennes par cluster pour chaque feature
unique_labels_sorted = sorted([l for l in set(labels) if l != -1])

cluster_profiles = []
for label in unique_labels_sorted:
    profile = df_with_clusters[df_with_clusters['Cluster'] == label][selected_cols[:10]].mean()
    cluster_profiles.append(profile)

if cluster_profiles:
    heatmap_data = pd.DataFrame(cluster_profiles)
    heatmap_data.index = [f'Cluster {i}' for i in unique_labels_sorted]
    
    # Normaliser pour la heatmap (scores Z)
    heatmap_normalized = (heatmap_data - heatmap_data.mean()) / heatmap_data.std()
    
    fig = px.imshow(
        heatmap_normalized.T,
        labels=dict(x="Cluster", y="Caractéristique", color="Score Z"),
        x=heatmap_normalized.index,
        y=heatmap_normalized.columns,
        color_continuous_scale='RdBu_r',
        aspect='auto',
        title='Heatmap des profils de clusters (valeurs standardisées)',
        height=600
    )
    
    fig.update_layout(
        font=dict(size=11),
        xaxis=dict(tickangle=0)
    )
    
    fig.show()
else:
    print("⚠️ Pas assez de clusters pour générer la heatmap")

## 🤖 Analyse IA - Insights automatiques

In [ ]:
def generate_ai_insights(df, labels, numeric_cols):
    """
    Génère des insights automatiques sur les clusters
    """
    insights = []
    unique_labels = sorted(set(labels))
    
    for label in unique_labels:
        if label == -1:
            insights.append({
                'cluster': 'Bruit/Outliers',
                'size': sum(labels == label),
                'description': 'Points considérés comme du bruit ou des outliers par l\'algorithme',
                'characteristics': []
            })
            continue
        
        cluster_data = df[labels == label][numeric_cols]
        cluster_size = len(cluster_data)
        
        # Calculer les statistiques
        characteristics = []
        for col in numeric_cols[:8]:  # Limiter à 8 colonnes
            mean_val = cluster_data[col].mean()
            overall_mean = df[col].mean()
            diff_pct = ((mean_val - overall_mean) / overall_mean) * 100 if overall_mean != 0 else 0
            
            if abs(diff_pct) > 20:  # Différence significative
                direction = "supérieur" if diff_pct > 0 else "inférieur"
                characteristics.append(f"{col}: {direction} de {abs(diff_pct):.1f}%")
        
        # Générer description
        if characteristics:
            description = f"Cluster de {cluster_size} éléments avec des caractéristiques distinctives."
        else:
            description = f"Cluster de {cluster_size} éléments avec un profil proche de la moyenne générale."
        
        insights.append({
            'cluster': f'Cluster {label}',
            'size': cluster_size,
            'description': description,
            'characteristics': characteristics[:5]  # Top 5 caractéristiques
        })
    
    return insights

# Générer les insights
insights = generate_ai_insights(df_processed, labels, selected_cols)

print("🤖 ANALYSE AUTOMATIQUE PAR IA")
print("=" * 70)

for insight in insights:
    print(f"\n{'='*70}")
    print(f"📊 {insight['cluster']}")
    print(f"{'='*70}")
    print(f"👥 Taille: {insight['size']} éléments ({insight['size']/len(labels)*100:.1f}%)")
    print(f"\n📝 Description:")
    print(f"   {insight['description']}")
    
    if insight['characteristics']:
        print(f"\n🎯 Caractéristiques principales:")
        for char in insight['characteristics']:
            print(f"   • {char}")
    print()

## 📝 Résumé de l'analyse

In [ ]:
print("📊 RÉSUMÉ DE L'ANALYSE DE CLUSTERING")
print("=" * 70)

print(f"\n🔧 CONFIGURATION")
print(f"   • Dataset: {df.shape[0]} lignes × {df.shape[1]} colonnes")
print(f"   • Features utilisées: {len(selected_cols)} colonnes")
print(f"   • Réduction de dimensionnalité: {REDUCTION_METHOD}")
print(f"   • Algorithme de clustering: {CLUSTERING_ALGORITHM}")

print(f"\n📊 RÉSULTATS")
print(f"   • Nombre de clusters: {len([l for l in set(labels) if l != -1])}")
if -1 in labels:
    print(f"   • Points de bruit/outliers: {sum(labels == -1)}")

if silhouette is not None:
    print(f"\n📈 MÉTRIQUES DE QUALITÉ")
    print(f"   • Silhouette Score: {silhouette:.4f}")
    print(f"   • Davies-Bouldin Index: {davies_bouldin:.4f}")
    print(f"   • Calinski-Harabasz Score: {calinski:.2f}")

print(f"\n💡 RECOMMANDATIONS")
if silhouette and silhouette > 0.5:
    print("   ✅ Les clusters sont bien définis et distincts")
    print("   ✅ La configuration actuelle donne de bons résultats")
elif silhouette and silhouette > 0.3:
    print("   ⚠️ Les clusters sont acceptables mais pourraient être améliorés")
    print("   💡 Essayez de modifier les hyperparamètres ou changez d'algorithme")
else:
    print("   ⚠️ La séparation des clusters est faible")
    print("   💡 Recommandations:")
    print("      - Essayez un nombre différent de clusters (k)")
    print("      - Testez un autre algorithme de clustering")
    print("      - Sélectionnez des features plus discriminantes")
    print("      - Ajustez les paramètres (epsilon, perplexity, etc.)")

print(f"\n{'='*70}")

## 💾 Export des résultats

In [ ]:
# Créer un DataFrame avec tous les résultats
results_df = df.copy()
results_df['Cluster'] = labels
results_df['Cluster_Name'] = [f'Cluster_{l}' if l != -1 else 'Outlier' for l in labels]

# Ajouter les composantes réduites
if N_COMPONENTS >= 2:
    results_df['Component_1'] = X_reduced[:, 0]
    results_df['Component_2'] = X_reduced[:, 1]
if N_COMPONENTS == 3:
    results_df['Component_3'] = X_reduced[:, 2]

# Télécharger le fichier
results_df.to_csv('clustering_results.csv', index=False)
files.download('clustering_results.csv')

print("✅ Résultats exportés dans 'clustering_results.csv'")
print(f"\n📊 Le fichier contient:")
print(f"   • Toutes les colonnes originales")
print(f"   • Labels des clusters")
print(f"   • Composantes réduites ({N_COMPONENTS}D)")

# Afficher un aperçu
print("\n📋 Aperçu des résultats:")
display(results_df.head(10))

## 🎓 Conclusion

Ce notebook vous a permis de :

✅ Charger et prétraiter vos données automatiquement  
✅ Appliquer plusieurs algorithmes de clustering  
✅ Utiliser différentes techniques de réduction de dimensionnalité  
✅ Évaluer la qualité avec plusieurs métriques  
✅ Visualiser les résultats de manière interactive  
✅ Obtenir des insights automatiques par IA  
✅ Exporter vos résultats  

### 💡 Prochaines étapes

1. **Optimisation**: Testez différents paramètres pour améliorer les métriques
2. **Features Engineering**: Créez de nouvelles variables pour mieux capturer les patterns
3. **Interprétation**: Utilisez les insights pour prendre des décisions business
4. **Validation**: Validez vos clusters avec des experts métier

---

**Challenge 2 : AI Clustering & Insight Dashboard** ✨  
Créé pour l'analyse intelligente de données